## Compare and contrast the performance of an SVM versus deep learning model. Build a binary classifier using an support vector machine (SVM) and deep learning model to predict whether or not a customer is likely to subscribe to a banking service after being targeted by telemarketing advertisements. Compare the predictive accuracy of either model.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
tele_df = pd.read_csv('bank_telemarketing.csv')
tele_df.head()

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


Make sure that none of our categorical variables require bucketing. Get the column names of categorical variables and check their number of unique values.

In [2]:
# Generate our categorical variable list
tele_cat = tele_df.dtypes[tele_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

Looking at the number of unique values for each categorical variable, there were no categories that require bucketing prior to encoding.

In [3]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(tele_cat)
encode_df.head()

,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


Once we have our encoded categorical variables, we need to merge our encoded columns back into our original DataFrame (as well as remove the unencoded columns) and replace the unencoded categorical variables with the encoded variables.

In [4]:
# Merge one-hot encoded features and drop the originals
tele_df = tele_df.merge(encode_df, left_index = True, right_index = True)
tele_df = tele_df.drop(tele_cat, 1)
tele_df.head()

,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [6]:
tele_df.columns

Index(['Age', 'Job_admin', 'Job_blue-collar', 'Job_entrepreneur',
       'Job_management', 'Job_other', 'Job_retired', 'Job_self-employed',
       'Job_services', 'Job_technician', 'Marital_Status_divorced',
       'Marital_Status_married', 'Marital_Status_single',
       'Education_Primary_Education', 'Education_Professional_Education',
       'Education_Secondary_Education', 'Education_Tertiary_Education',
       'Default_Credit_no', 'Default_Credit_yes', 'Housing_Loan_no',
       'Housing_Loan_yes', 'Personal_Loan_no', 'Personal_Loan_yes',
       'Subscribed_no', 'Subscribed_yes'],
      dtype='object')

Split data into the training and testing sets prior to standardization to not incorporate the testing values into the scale—testing values are only for evaluation. Perform the training/test split and standardize our numerical variables.

In [7]:
# Remove loan status target from features data
y = tele_df.Subscribed_yes.values
X = tele_df.drop(columns = ["Subscribed_no", "Subscribed_yes"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### After standardizing variables in both the training and testing data, our dataset is ready for both models.

Use the SVM’s linear kernel to try and create a linear boundary between the “Subscribed_yes” versus “Subscribed_no” groups.

In [8]:
# Create the SVM model
svm = SVC(kernel = 'linear')

In [11]:
# Train the model
svm.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test, y_pred):.3f}")

 SVM model accuracy: 0.873


Looking at the output of our SVM model, the model was able to correctly predict the customers who subscribed roughly 87% of the time, which is a respectable first-pass model. 

Next, we need to compile and evaluate our deep learning model. Again, we’ll use our typical binary classifier parameters:
* The first hidden layer will have an input_dim equal to the length of the scaled feature data X , 10 neuron units, and will use the relu activation function.
* The second hidden layer will have 5 neuron units and also will use the relu activation function.
* The loss function should be binary_crossentropy, using the adam optimizer.

NOTE: Unlike our basic neural network model, we don’t want to use two to three times the number of neurons as input variables—we don’t want our deeper layers to overfit the input data.

In [13]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, 
                             input_dim = number_input_features, 
                             activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, 
                             activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, 
                             activation = "sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss = "binary_crossentropy", 
           optimizer = "adam", 
           metrics = ["accuracy"])

In [14]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs = 50) 

# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 22857 samples
Epoch 1/50
22857/22857 [==============================] - 4s 175us/sample - loss: 0.4114 - accuracy: 0.8575
Epoch 2/50
22857/22857 [==============================] - 2s 93us/sample - loss: 0.3732 - accuracy: 0.8735
Epoch 3/50
22857/22857 [==============================] - 2s 89us/sample - loss: 0.3700 - accuracy: 0.8735
Epoch 4/50
22857/22857 [==============================] - 2s 89us/sample - loss: 0.3688 - accuracy: 0.8735
Epoch 5/50
22857/22857 [==============================] - 2s 91us/sample - loss: 0.3676 - accuracy: 0.8735
Epoch 6/50
22857/22857 [==============================] - 2s 98us/sample - loss: 0.3670 - accuracy: 0.8735
Epoch 7/50
22857/22857 [==============================] - 2s 91us/sample - loss: 0.3665 - accuracy: 0.8735
Epoch 8/50
22857/22857 [==============================] - 2s 91us/sample - loss: 0.3659 - accuracy: 0.8735
Epoch 9/50
22857/22857 [==============================] - 2s 95us/sample - loss: 0.3654 - accuracy: 0.8735
Epoch 10/50
2

Looking at the results of our comparative analysis, the SVM and deep learning models both achieved a predictive accuracy around 87%. 
* Additionally, both models take similar amounts of time to train on the input data. 
* The only noticeable difference between the two models is implementation.
    * The amount of code required to build and train the SVM is notably less than the comparable deep learning model. 
* As a result, many data scientists will prefer to use SVMs by default, then turn to deep learning models, as needed.